In [78]:
import andes
import ams
import cvxpy as cp
import numpy as np

# Load and formulate ams case

In [79]:
ssp = ams.load(ams.get_case('ieee14/ieee14_uced.xlsx'))

ssp.LDOPF.run()

LDOPF solved as optimal in 0.0111 seconds with exit code 0.


True

In [80]:
ssp.LDOPF.om

In [81]:
ssp.LDOPF.om.vars

OrderedDict([('pg', Variable((5,))),
             ('pn', Variable((14,))),
             ('qg', Variable((5,))),
             ('qn', Variable((14,))),
             ('vsq', Variable((14,))),
             ('pl', Variable((20,))),
             ('ql', Variable((20,)))])

In [82]:
cvx_mdl = ssp.LDOPF.om.mdl

cvx_mdl.solve()

69.55849510545293

# Modify ams cvxpy model

### define new variables

In [83]:
# define new variables

lb = np.array([0, 0])
ub = np.array([1, 1])

x = cp.Variable(2, name="x")

# x = cp.Variable(2, name='x', lb=np.array([0, 0]), ub=np.array([1, 1]))

### modify constraints

In [84]:
# define new constraints

constraints_new = [x >= lb, x <= ub]

In [85]:
# extract constraints from ams model

constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]
constriants_ams

[Inequality(Constant(CONSTANT, NONNEGATIVE, (5,))),
 Inequality(Variable((5,))),
 Inequality(Constant(CONSTANT, NONPOSITIVE, (5,))),
 Inequality(Variable((5,))),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, (14,)), Constant(CONSTANT, ZERO, ())),
 Inequality(Expression(AFFINE, UNKNOWN, (20,))),
 Inequality(Expression(AFFINE, UNKNOWN, (20,))),
 Inequality(Expression(AFFINE, UNKNOWN, (14,))),
 Inequality(Expression(AFFINE, UNKNOWN, (14,))),
 Equality(Expression(AFFINE, UNKNOWN, (14,)), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, (20,)), Constant(CONSTANT, ZERO, ()))]

In [86]:
# formulated new constraints

constraints_new += constriants_ams
constraints_new

[Inequality(Constant(CONSTANT, ZERO, (2,))),
 Inequality(Variable((2,))),
 Inequality(Constant(CONSTANT, NONNEGATIVE, (5,))),
 Inequality(Variable((5,))),
 Inequality(Constant(CONSTANT, NONPOSITIVE, (5,))),
 Inequality(Variable((5,))),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, (14,)), Constant(CONSTANT, ZERO, ())),
 Inequality(Expression(AFFINE, UNKNOWN, (20,))),
 Inequality(Expression(AFFINE, UNKNOWN, (20,))),
 Inequality(Expression(AFFINE, UNKNOWN, (14,))),
 Inequality(Expression(AFFINE, UNKNOWN, (14,))),
 Equality(Expression(AFFINE, UNKNOWN, (14,)), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, (20,)), Constant(CONSTANT, ZERO, ()))]

### define new objective function

In [87]:
obj = cp.Minimize(cp.sum(x))

In [88]:
x

Variable((2,))

In [95]:
np.array([1, 2]).shape

(2,)

In [97]:
temp = x @ np.array([1, 2])
temp

Expression(AFFINE, UNKNOWN, ())

In [89]:
obj = cp.Minimize(x @ np.array([1, 2]))

### formulate new cvxpy problem

In [90]:
prob = cp.Problem(obj, constraints_new)

In [91]:
prob.solve()

1.5578968827277052e-11

# Check optimization results

In [92]:
# optimization status

if prob.status == cp.OPTIMAL:
    print("Optimal solution found.")
elif prob.status == cp.INFEASIBLE:
    print("The problem is infeasible.")
elif prob.status == cp.UNBOUNDED:
    print("The problem is unbounded.")
else:
    print("Solver did not converge or encountered an issue.")

Optimal solution found.


In [93]:
# objective value

optimal_objective_value = prob.value

print("Optimal Objective Value:", optimal_objective_value)

Optimal Objective Value: 1.5578968827277052e-11


In [94]:
# decision variables values

pg_optimal = prob.pg.value

print("Optimal values of 'pg':", pg_optimal)

AttributeError: 'Problem' object has no attribute 'pg'

In [ ]:
# decision variables values

pg_optimal = ssp.LDOPF.om.pg.value

print("Optimal values of 'pg':", pg_optimal)

Optimal values of 'pg': [0.27022003 0.27022003 0.27022003 0.27022003 1.15611989]
